In [1]:
import pandas as pd
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder.master("local[4]").appName("ClusterShop").getOrCreate()

# Read the Excel file using Pandas
df_pandas = pd.read_excel("Online Retail.xlsx",na_values='')

# Convert the Pandas DataFrame to PySpark DataFrame
df_spark = spark.createDataFrame(df_pandas)

23/04/15 15:21:08 WARN Utils: Your hostname, khaldon-LENOVO resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface wlp0s20f3)
23/04/15 15:21:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/15 15:21:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Show the DataFrame
df_spark.show()

23/04/15 15:23:55 WARN TaskSetManager: Stage 0 contains a task of very large size (8485 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [3]:
# Describe the DataFrame
df_spark.describe().show()


23/04/15 15:23:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/04/15 15:23:58 WARN TaskSetManager: Stage 1 contains a task of very large size (8485 KiB). The maximum recommended task size is 1000 KiB.


+-------+------------------+------------------+--------------------+------------------+-----------------+----------+-----------+
|summary|         InvoiceNo|         StockCode|         Description|          Quantity|        UnitPrice|CustomerID|    Country|
+-------+------------------+------------------+--------------------+------------------+-----------------+----------+-----------+
|  count|            541909|            541909|              541909|            541909|           541909|    541909|     541909|
|   mean|  559965.752026781|27623.240210938104|                 NaN|  9.55224954743324|4.611113626088897|       NaN|       null|
| stddev|13428.417280798658| 16799.73762842766|                 NaN|218.08115785023472|96.75985306117991|       NaN|       null|
|    min|            536365|             10002| 4 PURPLE FLOCK D...|            -80995|        -11062.06|   12346.0|  Australia|
|    max|           C581569|                 m|   wrongly sold sets|             80995|          

In [4]:
import math
# count the number of missing values in the CustomerID column
df_spark.rdd.map(lambda x: 1 if math.isnan( x['CustomerID'])  else 0).sum()

# filter out the missing values using map 
df_spark_filtered = df_spark.rdd.map(lambda x: x if not math.isnan( x['CustomerID'])  else None).filter(lambda x: x is not None).toDF()



23/04/15 15:24:05 WARN TaskSetManager: Stage 4 contains a task of very large size (8485 KiB). The maximum recommended task size is 1000 KiB.
23/04/15 15:24:10 WARN TaskSetManager: Stage 5 contains a task of very large size (8485 KiB). The maximum recommended task size is 1000 KiB.


In [5]:
df_spark_filtered.describe().show()

23/04/15 15:24:10 WARN TaskSetManager: Stage 6 contains a task of very large size (8485 KiB). The maximum recommended task size is 1000 KiB.


+-------+-----------------+------------------+--------------------+-----------------+------------------+------------------+-----------+
|summary|        InvoiceNo|         StockCode|         Description|         Quantity|         UnitPrice|        CustomerID|    Country|
+-------+-----------------+------------------+--------------------+-----------------+------------------+------------------+-----------+
|  count|           406829|            406829|              406829|           406829|            406829|            406829|     406829|
|   mean|560617.1266447864|27430.341352504624|                null|12.06130339774205|3.4604710185376426|15287.690570239585|       null|
| stddev|13106.16769474081|16403.570452626223|                null|248.6933700188252| 69.31516172321443|1713.6003033215966|       null|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|           -80995|               0.0|           12346.0|  Australia|
|    max|          C581569|              POST|ZI

In [6]:
# convert the InvoiceDate column to a date
from pyspark.sql.functions import to_date
df_spark_filtered = df_spark_filtered.withColumn("InvoiceDate", to_date(df_spark_filtered.InvoiceDate, 'MM/dd/yyyy'))
    

In [7]:

from pyspark.sql.functions import col
from pyspark.sql.functions import round
from pyspark.sql.functions import month

# create a new column called TotalCost
df_spark_filtered = df_spark_filtered.withColumn("TotalCost", col("Quantity") * col("UnitPrice"))

# round the TotalCost column to 2 decimal places
df_spark_filtered = df_spark_filtered.withColumn("TotalCost", round(df_spark_filtered.TotalCost, 2))

# create a new column called InvoiceMonth
df_spark_filtered = df_spark_filtered.withColumn("InvoiceMonth", month(df_spark_filtered.InvoiceDate))


df_spark_filtered.show()

23/04/15 15:24:26 WARN TaskSetManager: Stage 9 contains a task of very large size (8485 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+--------------------+--------+-----------+---------+----------+--------------+---------+------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|TotalCost|InvoiceMonth|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+---------+------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-12-01|     2.55|   17850.0|United Kingdom|     15.3|          12|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-12-01|     3.39|   17850.0|United Kingdom|    20.34|          12|
|   536365|   84406B|CREAM CUPID HEART...|       8| 2010-12-01|     2.75|   17850.0|United Kingdom|     22.0|          12|
|   536365|   84029G|KNITTED UNION FLA...|       6| 2010-12-01|     3.39|   17850.0|United Kingdom|    20.34|          12|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6| 2010-12-01|     3.39|   17850.0|United Kingdom|    20.34|          12|
|   536365|    2

In [8]:
# Import necessary functions
from pyspark.sql.functions import count, mean, sum

# Define a function to aggregate the data by customer ID and invoice month
def aggregate_data_by_customer(row):
    # Extract the customer ID and invoice month from the row
    customerID, invoiceMonth, invoiceNo, totalCost = row["CustomerID"], row["InvoiceMonth"], row["InvoiceNo"], row["TotalCost"]
    # Convert the total cost to a float
    totalCost = float(totalCost)
    # Return a tuple containing the customer ID, invoice month, and a dictionary with the aggregated values
    return ((customerID, invoiceMonth), {"TotalPurchases": 1, "TotalRevenue": totalCost, "AvgRevenuePerPurchase": totalCost})

# Define a function to combine the aggregated data for each customer
def combine_customer_data(data1, data2):
    # Combine the dictionaries for the two sets of aggregated data
    combined_data = {k: data1.get(k, 0) + data2.get(k, 0) for k in set(data1) | set(data2)}
    # Return the combined data
    return combined_data

# Map the data using the aggregate_data_by_customer() function
mapped_data = df_spark_filtered.rdd.map(aggregate_data_by_customer)

# Reduce the data using the combine_customer_data() function
reduced_data = mapped_data.reduceByKey(combine_customer_data)

# Convert the reduced data to a DataFrame
df_customer = reduced_data.map(lambda x: (x[0][0], x[0][1], x[1]["TotalPurchases"], x[1]["TotalRevenue"], x[1]["AvgRevenuePerPurchase"])).toDF(["CustomerID", "InvoiceMonth", "TotalPurchases", "TotalRevenue", "AvgRevenuePerPurchase"])

# show the data frame   
df_customer.show()  

23/04/15 15:24:28 WARN TaskSetManager: Stage 10 contains a task of very large size (8485 KiB). The maximum recommended task size is 1000 KiB.


+----------+------------+--------------+------------------+---------------------+
|CustomerID|InvoiceMonth|TotalPurchases|      TotalRevenue|AvgRevenuePerPurchase|
+----------+------------+--------------+------------------+---------------------+
|   17850.0|          12|           297| 5391.210000000009|    5391.210000000009|
|   16098.0|          12|            12|430.59999999999997|   430.59999999999997|
|   18074.0|          12|            13|             489.6|                489.6|
|   16250.0|          12|            14|226.14000000000001|   226.14000000000001|
|   15862.0|          12|           111| 527.3799999999999|    527.3799999999999|
|   16218.0|          12|            28|471.29999999999995|   471.29999999999995|
|   12838.0|          12|            59|390.78999999999985|   390.78999999999985|
|   13758.0|          12|            24|            718.85|               718.85|
|   13694.0|          12|            70|            6016.2|               6016.2|
|   16210.0|    